#### Send ontology file for importation

In [1]:
import requests

app_api_host = 'https://api.app.lettria.com'
file_name = 'ontology.owl'
file_location = './data/'
api_route = app_api_host + '/app/knowledge/import'
jwt_token_file = 'jwt_token.txt'
jwt_token = open(jwt_token_file, 'r').read()
knowlede_id = None

with open(file_location + file_name, 'rb') as file:
    response = requests.post(
        api_route,
        files=[('file', ( file_name, file, "application/rdf+xml"))],
        headers={"Authorization":"LettriaProKey " + jwt_token}
    )
    print(response.json())
    knowlede_id = response.json()['knowledgeId']

{'message': 'ok', 'knowledgeId': '6573461c911ed5002428cd10'}


#### Polling to wait for the end of the import

In [2]:
import time
import requests

api_route = app_api_host + '/app/knowledge/read'
importation_is_finished = False

knowledge = None
importation_status = None
while not importation_is_finished:
    response = requests.post(
        api_route,
        headers={"Authorization":"LettriaProKey " + jwt_token}
    ).json()
    imported_knowledge = next((item for item in response['documents'] if item["_id"] == knowlede_id), None)
    importation_status = imported_knowledge.get('importationStatus')
    print(importation_status + "      ", end='\r')
    importation_is_finished = importation_status != 'PROCESSING'
    time.sleep(1)
if importation_status != 'SUCCESS':
    print('Importation failed')
    exit(1)